# Step 1: Install Required Libraries
Run this cell to install necessary libraries.
!pip install speechrecognition gtts transformers torch librosa


In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os
from transformers import pipeline
import time

# Step 2: Initialize Translator
Run this cell to load the translation models.

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")  # English to Spanish
translator_reverse = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")  # Spanish to English

# Step 3: Define Speech-to-Text Function
Run this cell to define the function for speech recognition.

In [ ]:
def speech_to_text(language="en-US"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print(f"Listening for {language}... Speak now!")
        try:
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio, language=language)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            print("Could not understand the audio.")
            return ""
        except sr.RequestError as e:
            print(f"Error with speech recognition service: {e}")
            return ""

# Step 4: Define Translation Function
Run this cell to define the function for text translation.

In [ ]:
def translate_text(text, target_language="es"):
    if target_language == "es":
        result = translator(text)[0]['translation_text']
    else:
        result = translator_reverse(text)[0]['translation_text']
    print(f"Translated Text ({target_language}): {result}")
    return result

# Step 5: Define Text-to-Speech Function
Run this cell to define the function for text-to-speech conversion.

In [ ]:
def text_to_speech(text, language="es"):
    tts = gTTS(text, lang=language)
    tts.save("output.mp3")
    os.system("start output.mp3")  # For Windows

# Step 6: Real-Time Conversation Loop
Run this cell to define the conversation loop function.

In [ ]:
def real_time_conversation():
    print("Starting real-time conversation. Press Ctrl+C to stop.")
    while True:
        try:
            # English Speaker
            english_text = speech_to_text(language="en-US")
            if english_text:
                spanish_translation = translate_text(english_text, target_language="es")
                text_to_speech(spanish_translation, language="es")
            # Spanish Speaker
            spanish_text = speech_to_text(language="es-ES")
            if spanish_text:
                english_translation = translate_text(spanish_text, target_language="en")
                text_to_speech(english_translation, language="en")
        except KeyboardInterrupt:
            print("Conversation ended.")
            break

# Step 7: Accuracy Evaluation Function
Run this cell to define a function to evaluate accuracy.

In [ ]:
def evaluate_accuracy(test_data, target_language):
    correct = 0
    total = len(test_data)
    for original_text, expected_translation in test_data:
        if target_language == "es":
            predicted_translation = translate_text(original_text, target_language="es")
        else:
            predicted_translation = translate_text(original_text, target_language="en")
        if predicted_translation.strip().lower() == expected_translation.strip().lower():
            correct += 1
        else:
            print(f"Error: Original: {original_text}, Predicted: {predicted_translation}, Expected: {expected_translation}")
    accuracy = (correct / total) * 100
    print(f"Model Accuracy: {accuracy:.2f}%")
    return accuracy

# Example Usage for Accuracy Evaluation
Use this function to test the accuracy of the model on predefined data.

In [ ]:
def example_accuracy_evaluation():
    test_data = [
        ("Hello, how are you?", "Hola, ¿cómo estás?"),
        ("I am learning Python.", "Estoy aprendiendo Python."),
        ("What is your name?", "¿Cuál es tu nombre?"),
    ]
    evaluate_accuracy(test_data, target_language="es")

# Step 8: Run the Program
Run this cell to start the real-time conversation or test accuracy.

In [ ]:
if __name__ == "__main__":
    # Uncomment below to start real-time conversation
    # real_time_conversation()
    # Uncomment below to test accuracy evaluation
    # example_accuracy_evaluation()